In [3]:
import os
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns

file_name = '60yWAC_'

pred_a = pd.read_csv(file_name + 'A.csv')
pred_a['date'] = pd.to_datetime(pred_a['date_forecast'])
pred_b = pd.read_csv(file_name + 'B.csv')
pred_b['date'] = pd.to_datetime(pred_b['date_forecast'])
pred_c = pd.read_csv(file_name + 'C.csv')
pred_c['date'] = pd.to_datetime(pred_c['date_forecast'])

test = pd.read_csv('data/test.csv')
test['time'] = pd.to_datetime(test['time'])

C:\Users\47901\anaconda3\lib\site-packages\pandas\_testing.py:24: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  import pandas._libs.testing as _testing


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [11]:
submission = pd.DataFrame(columns=['prediction'])

for val in pred_a['date']:
    if test['time'].eq(val).any():
        row = pred_a.loc[pred_a['date'] == val]
        submission = submission.append({'prediction': float(row['predicted'])}, ignore_index=True)

for val in pred_b['date']:
    if test['time'].eq(val).any():
        row = pred_b.loc[pred_a['date'] == val]
        submission = submission.append({'prediction': float(row['predicted'])}, ignore_index=True)


for val in pred_c['date']:
    if test['time'].eq(val).any():
        row = pred_c.loc[pred_a['date'] == val]
        submission = submission.append({'prediction': float(row['predicted'])}, ignore_index=True)
        

submission['prediction'] = submission['prediction'].where(submission['prediction'] >= 0, 0)   

submission.to_csv('submission_' + file_name + 'attempt.csv')

In [7]:
sub = pd.read_csv('sub_h2o.csv')

sub['prediction'] = sub['prediction'].where(sub['prediction'] >= 0, 0)
sub = sub.drop(columns='id')

sub.to_csv('no_minus.csv')

In [50]:
def average_two_csv(filename1, filename2):
    file1 = pd.read_csv(filename1)
    file2 = pd.read_csv(filename2)

    merged_df = file1.merge(file2, on='id', suffixes=('_df1', '_df2'))

    # Compute the average of the 'prediction' columns
    merged_df['prediction'] = (merged_df['prediction_df1'] + merged_df['prediction_df2']) / 2

    # Keep only the 'id' and the 'prediction' columns
    result = merged_df[['id', 'prediction']]

    # Remember ti write id on the top of the column when delivering :)
    result = result.drop(columns='id')

    result.to_csv(filename1.split('.')[0] + '_merged_' + filename2)
    print('Wrote this file: ' + filename1.split('.')[0] + '_merged_' + filename2)

    
average_two_csv('lgbm.csv', 'average_of_two_best.csv')

Wrote this file: lgbm_merged_average_of_two_best.csv


In [3]:
def calcGrade(VT_defeated=5):
    number_score = 41 + (100-41)/(10-1) * (VT_defeated-1)
    print(number_score)
    
calcGrade()

67.22222222222223


In [2]:
import utils

In [40]:
train_a, test = utils.preprocess_category('C')
test_a = test.drop(columns=['date_calc', 'date_forecast'])

Total data points: 26095
Data points to be removed: 2110


In [41]:
train_a_Y = train_a['pv_measurement']
train_a_X = train_a.drop(columns=['pv_measurement'])
train_a_X = train_a_X.drop(columns=['date_calc', 'date_forecast', 'time'])

In [42]:
import lightgbm as lgb
import optuna
from sklearn.model_selection import cross_val_score

In [43]:
def clean_feature_name(name):
    # Replace any special characters you want here
    return name.replace(':', '_').replace('{', '_').replace('}', '_').replace('"', '_')

train_a_X.columns = [clean_feature_name(col) for col in train_a_X.columns]
test_a.columns = [clean_feature_name(col) for col in test_a.columns]
print(train_a_X)


        absolute_humidity_2m_gm3  air_density_2m_kgm3  ceiling_height_agl_m  \
0                            6.7                1.223           2122.699951   
1                            6.7                1.222           2231.800049   
2                            6.6                1.220           2341.800049   
3                            6.5                1.218           2452.699951   
4                            6.5                1.217           2564.500000   
...                          ...                  ...                   ...   
110745                       4.4                1.279           1446.599976   
110746                       4.4                1.280           1309.300049   
110747                       4.4                1.280           1171.900024   
110748                       4.4                1.280           1034.599976   
110749                       4.4                1.280            897.200012   

        clear_sky_energy_1h_J  clear_sky_rad_W  clo

In [44]:
def objective(trial):
    dtrain = lgb.Dataset(train_a_X, label=train_a_Y)
    param = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, step=0.01),
        "boosting_type": trial.suggest_categorical("boosting_type", ['dart', 'gbdt']),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "verbose": -1
    }

    model = lgb.LGBMRegressor(**param)
    model.fit(train_a_X, train_a_Y)

#     result = cross_val_score(model, train_X_std.values, train_Y.values.ravel(), cv=5, scoring="neg_mean_squared_log_error")
    result = cross_val_score(model, train_a_X.values, train_a_Y.values.ravel(), cv=5, scoring="neg_mean_squared_error")


    tot = 0
    for part in result:
        tot = tot + (-part)
    accuracy = tot/len(result)

    return accuracy


In [45]:

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-11-02 12:44:36,796] A new study created in memory with name: no-name-b06c2f0d-2fa0-44f7-ae89-90ccf4b0a095


[LightGBM] [Warning] feature_fraction is set=0.8732360422452954, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8732360422452954
[LightGBM] [Warning] lambda_l1 is set=7.849043973749655e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.849043973749655e-05
[LightGBM] [Warning] bagging_fraction is set=0.6104704976410541, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6104704976410541
[LightGBM] [Warning] lambda_l2 is set=9.211666463796506e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.211666463796506e-08
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1
[LightGBM] [Warning] feature_fraction is set=0.8732360422452954, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8732360422452954
[LightGBM] [Warning] lambda_l1 is set=7.849043973749655e-05,

[I 2023-11-02 12:44:46,885] Trial 0 finished with value: 4033.70601405888 and parameters: {'learning_rate': 0.09999999999999999, 'boosting_type': 'gbdt', 'lambda_l1': 7.849043973749655e-05, 'lambda_l2': 9.211666463796506e-08, 'num_leaves': 121, 'feature_fraction': 0.8732360422452954, 'bagging_fraction': 0.6104704976410541, 'bagging_freq': 2, 'min_child_samples': 11}. Best is trial 0 with value: 4033.70601405888.


[LightGBM] [Warning] feature_fraction is set=0.7401498688251538, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7401498688251538
[LightGBM] [Warning] lambda_l1 is set=4.444706119559683e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.444706119559683e-06
[LightGBM] [Warning] bagging_fraction is set=0.49288848633652144, subsample=1.0 will be ignored. Current value: bagging_fraction=0.49288848633652144
[LightGBM] [Warning] lambda_l2 is set=1.3661956976135392e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.3661956976135392e-08
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1
[LightGBM] [Warning] feature_fraction is set=0.7401498688251538, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7401498688251538
[LightGBM] [Warning] lambda_l1 is set=4.444706119559683e

[I 2023-11-02 12:44:55,629] Trial 1 finished with value: 3699.432493495156 and parameters: {'learning_rate': 0.03, 'boosting_type': 'gbdt', 'lambda_l1': 4.444706119559683e-06, 'lambda_l2': 1.3661956976135392e-08, 'num_leaves': 79, 'feature_fraction': 0.7401498688251538, 'bagging_fraction': 0.49288848633652144, 'bagging_freq': 1, 'min_child_samples': 92}. Best is trial 1 with value: 3699.432493495156.


...

[LightGBM] [Warning] feature_fraction is set=0.8438243559243086, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8438243559243086
[LightGBM] [Warning] lambda_l1 is set=1.920388474046705, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.920388474046705
[LightGBM] [Warning] bagging_fraction is set=0.9456822772312813, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9456822772312813
[LightGBM] [Warning] lambda_l2 is set=1.6248843369602095e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.6248843369602095e-05
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1
[LightGBM] [Warning] feature_fraction is set=0.8438243559243086, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8438243559243086
[LightGBM] [Warning] lambda_l1 is set=1.920388474046705, reg_alpha

[I 2023-11-02 13:03:16,111] Trial 98 finished with value: 3547.37908918443 and parameters: {'learning_rate': 0.04, 'boosting_type': 'gbdt', 'lambda_l1': 1.920388474046705, 'lambda_l2': 1.6248843369602095e-05, 'num_leaves': 13, 'feature_fraction': 0.8438243559243086, 'bagging_fraction': 0.9456822772312813, 'bagging_freq': 7, 'min_child_samples': 10}. Best is trial 43 with value: 3533.207707024662.


[LightGBM] [Warning] feature_fraction is set=0.8958881232623127, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8958881232623127
[LightGBM] [Warning] lambda_l1 is set=2.5836290132596904, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.5836290132596904
[LightGBM] [Warning] bagging_fraction is set=0.9456360458759527, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9456360458759527
[LightGBM] [Warning] lambda_l2 is set=1.3608509743491159e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.3608509743491159e-05
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1
[LightGBM] [Warning] feature_fraction is set=0.8958881232623127, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8958881232623127
[LightGBM] [Warning] lambda_l1 is set=2.5836290132596904, reg_al

[I 2023-11-02 13:03:28,125] Trial 99 finished with value: 3707.198830164819 and parameters: {'learning_rate': 0.04, 'boosting_type': 'gbdt', 'lambda_l1': 2.5836290132596904, 'lambda_l2': 1.3608509743491159e-05, 'num_leaves': 44, 'feature_fraction': 0.8958881232623127, 'bagging_fraction': 0.9456360458759527, 'bagging_freq': 2, 'min_child_samples': 14}. Best is trial 43 with value: 3533.207707024662.


Number of finished trials: 100
Best trial:
  Value: 3533.207707024662
  Params: 
    learning_rate: 0.04
    boosting_type: gbdt
    lambda_l1: 0.02881645852127883
    lambda_l2: 0.3031128635062045
    num_leaves: 15
    feature_fraction: 0.9285965761166466
    bagging_fraction: 0.9715141232509082
    bagging_freq: 7
    min_child_samples: 19


In [46]:
param = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "learning_rate": 0.04,
        "boosting_type":  'gbdt',
        "lambda_l1": 0.02881645852127883,
        "lambda_l2": 0.3031128635062045,
        "num_leaves": 15,
        "feature_fraction": 0.9285965761166466,
        "bagging_fraction": 0.9715141232509082,
        "bagging_freq": 7,
        "min_child_samples": 19,
        "verbose": -1
    }

model = lgb.LGBMRegressor(**param)
model.fit(train_a_X, train_a_Y)
prediction = model.predict(test_a)
preds = pd.DataFrame()
preds['date_forecast'] = test['date_forecast']
preds['prediction'] = np.asarray(prediction)

preds['prediction'] = preds['prediction'].where(preds['prediction'] >= 0, 0)

preds.to_csv('lgbm_c.csv', index=False)

[LightGBM] [Warning] feature_fraction is set=0.9285965761166466, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9285965761166466
[LightGBM] [Warning] lambda_l1 is set=0.02881645852127883, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.02881645852127883
[LightGBM] [Warning] bagging_fraction is set=0.9715141232509082, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9715141232509082
[LightGBM] [Warning] lambda_l2 is set=0.3031128635062045, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3031128635062045
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1
